In [ ]:
import sqlite3
import pandas as pd
import math
products=pd.read_excel("sales_original.xlsx",
                        sheet_name="products")
stock=pd.read_excel("sales_original.xlsx",
                        sheet_name="stock")
assessment=pd.read_excel("sales_original.xlsx",
                        sheet_name="assessment")
invoices=pd.read_excel("sales_original.xlsx",
                        sheet_name="invoices")
customers=pd.read_excel("sales_original.xlsx",
                        sheet_name="customers")                                                
conn=sqlite3.connect("df.db")
c=conn.cursor()

In [ ]:
products1=products.drop_duplicates(subset="ASIN")
products1=products1.dropna()
products1=products1[products1["ASIN"]!="Null"]
products1=products1.groupby(["ASIN","product_type"]).sum()
products1=products1.reset_index()
stock1=stock.drop_duplicates(subset="ASIN")
stock1=stock1.dropna()
stock1=stock1[stock1["ASIN"]!="Null"]
assessment1=assessment.drop_duplicates(subset="ASIN")
assessment1=assessment1.dropna()
assessment1=assessment1[assessment1["ASIN"]!="Null"]
invoices1=invoices.drop_duplicates()
invoices1=invoices1[invoices1["ASIN"]!="Null"]
customers1=customers.drop_duplicates(subset="CustomerID")
customers1=customers1.dropna()
customers1=customers1[customers1["CustomerID"]!="Null"]

In [ ]:
products.nunique()

In [ ]:
products1.info()

In [ ]:
c.execute("""CREATE TABLE IF NOT EXISTS products
        (ASIN PRIMARY KEY,
        title,
        product_type)
        """)
products1.to_sql("products", conn, if_exists="replace", index=False)
c.execute("""CREATE TABLE IF NOT EXISTS stock
        (StockCode,
        ASIN PRIMARY KEY)
        """)
stock1.to_sql("stock", conn, if_exists="replace", index=False)
c.execute("""CREATE TABLE IF NOT EXISTS assessment
        (rating,
        review_count,
        ASIN PRIMARY KEY)
        """)
assessment1.to_sql("assessment", conn, if_exists="replace", index=False)
c.execute("""CREATE TABLE IF NOT EXISTS invoices
        (InvoiceNo,
        ASIN PRIMARY KEY,
        Quantity,
        price,
        total_sale,
        invoice_date,
        invoice_time,
        CustomerID)
        """)
invoices1.to_sql("invoices", conn, if_exists="replace", index=False)
c.execute("""CREATE TABLE IF NOT EXISTS customers
        (CustomerID
        Country)
        """)
customers1.to_sql("customers", conn, if_exists="replace", index=True)

In [ ]:
# Average ratings for each product type
c.execute("""SELECT p.product_type, ROUND(AVG(a.rating),2) 
        FROM products AS p
        JOIN assessment AS a
        ON a.ASIN=p.ASIN
        GROUP BY p.product_type
        ORDER BY AVG(a.rating) DESC
        """).fetchall()

In [ ]:
# 5 countries generating max revenue, excluding host
c.execute("""SELECT c.country, ROUND(SUM(i.total_sale),2)
        FROM customers AS c
        JOIN invoices AS i
        ON i.CustomerID=c.CustomerID
        WHERE c.country IS NOT "Germany"
        GROUP BY c.country
        ORDER BY SUM(i.total_sale) DESC
        LIMIT 5
        """).fetchall()

In [ ]:
# 3 products in each segment with the highest rating
c.execute("""WITH cte AS (
        SELECT * 
        FROM products AS p
        JOIN assessment AS a
        ON a.ASIN=p.ASIN)
        SELECT DISTINCT title, rating, product_type
        FROM
        (
            SELECT cte.*, ROW_NUMBER() OVER (
                PARTITION BY product_type ORDER BY rating DESC) rnk
            FROM cte)
        WHERE rnk<=3
        ORDER BY product_type, rating DESC
        """).fetchall()

In [ ]:
# Check if the above items are also the most reviewed
# They are not 
c.execute("""WITH cte AS (
        SELECT * 
        FROM products AS p
        JOIN assessment AS a
        ON a.ASIN=p.ASIN)
        SELECT DISTINCT title, review_count, product_type
        FROM
        (
            SELECT cte.*, ROW_NUMBER() OVER (
            PARTITION BY product_type ORDER BY review_count DESC) rnk
            FROM cte)
        WHERE rnk<=3
        ORDER BY product_type, review_count DESC
        """).fetchall()

In [ ]:
# 3 best selling products in each category
c.execute("""WITH cte1 AS (
        SELECT ASIN, SUM(Quantity) AS sold
        FROM invoices AS i
        GROUP BY ASIN),
        cte2 AS (
            SELECT p.product_type, cte1.ASIN, p.title,
            RANK () OVER (PARTITION by product_type ORDER BY sold DESC) rnk
            FROM cte1
            LEFT JOIN products AS p
            ON p.ASIN=cte1.ASIN)
        SELECT * 
        FROM cte2
        WHERE rnk BETWEEN 0 AND 3
        """).fetchall()

In [ ]:
# Get the first and second worst selling products in each category
# Multiple items have sale quantity 1, so barring any other filters,
# we can not select only 2 for each category
c.execute("""WITH cte1 AS (
        SELECT ASIN, SUM(Quantity) AS sold
        FROM invoices AS i
        GROUP BY ASIN),
        cte2 AS (
            SELECT p.product_type, cte1.ASIN, p.title,
            RANK () OVER (PARTITION by product_type ORDER BY sold) rnk
            FROM cte1
            LEFT JOIN products AS p
            ON p.ASIN=cte1.ASIN)
        SELECT * 
        FROM cte2
        WHERE rnk BETWEEN 0 AND 2
        ORDER BY product_type
        """).fetchall()

In [46]:
# Find the unique customers per month
c.execute("""WITH cte AS
                (SELECT DISTINCT CustomerID, 
                strftime("%m",invoice_date) AS month
                FROM invoices
                WHERE strftime("%Y",invoice_date)="2019")
        SELECT COUNT(c.CustomerID), month
        FROM cte
        JOIN Customers AS c
        ON cte.CustomerID=c.CustomerID
        GROUP BY month
        """).fetchall()

[(741, '01'),
 (758, '02'),
 (974, '03'),
 (851, '04'),
 (1056, '05'),
 (991, '06'),
 (948, '07'),
 (934, '08'),
 (1263, '09'),
 (1363, '10'),
 (1664, '11'),
 (615, '12')]

In [48]:
customers1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4339 entries, 0 to 552748
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   CustomerID  4339 non-null   object
 1   Country     4339 non-null   object
dtypes: object(2)
memory usage: 101.7+ KB
